In [2]:
import pandas as pd
import json, os
from time import strftime
import numpy as np

In [3]:
BAD_OWNERS = ['SENTER', 'RAILWAY', 'CITY', 'GOVERNMENT', 'GOVT', 'STATE OF', 'PIPE', 'CEMETERY', 'SCHOOL', 'TOWER', 'DISTRICT', 'SYSTEM']
BAD_ZONING = ['Commercial', 'Public Street', 'Utility Use']
OUTPUT_DIR = '../output/'

In [4]:
def split_address(address):
    *street, town, area = address.split(',')
    state, Zip = area.split()
    street, town, state, Zip = (s.strip() for s in (','.join(street), town, state, Zip))
    Zip = Zip[:5]
    return street, town, state, Zip

def good_address(address_list):
    good_flags = []
    for address in address_list:
        try:
            split_address(address)
            good_flags.append(True)
        except:
            good_flags.append(False)
    return good_flags

def df_to_click2mail(df):
    ad_columns     = ['Address', 'City', 'State', 'Zip']
    output         = pd.DataFrame(map(split_address, df.owner_address.values), columns=ad_columns)
    output['Name'] = [name[:-2] if name.endswith(' &') else name for name in df.owner_name]
    output.drop_duplicates(subset='Name', inplace=True)
    
    return output[['Name', *ad_columns]]

def remove_bad_owners(df, col='Name'):
    bad_owners_str = '|'.join(BAD_OWNERS)
    return df[~df[col].str.contains(bad_owners_str, case=False)]

**Tax penalty 1..20 acres and Wylie 1..10 acres, absentee only**

In [14]:
# Tax delinquent land with penalty

min_acres, max_acres = 0.9, 20.1

df      = pd.read_json(f'{OUTPUT_DIR}/output_taylor/output_taylor.json')
df      = df[good_address(df.owner_address.values)]
df      = df[df.absentee & df.empty_land & ~df.inactive & df.recent_penalty]
df      = df[~df.zoning.isin(BAD_ZONING)]
col_set = set(df.columns)-{'absentee', 'empty_land', 'inactive', 'land_dict', 'property_use'}
df      = df[[*col_set]]
df      = df.sort_values(by='recent_penalty', ascending=False)
df      = df[(df.land_area <= max_acres) & (min_acres <= df.land_area)]

df1 = df

In [15]:
# Small empty land in Wylie ISD

min_acres, max_acres = 0.9, 10.1

df      = pd.read_json(f'{OUTPUT_DIR}/output_taylor/output_taylor.json')
df      = df[good_address(df.owner_address.values)]
df      = df[df.absentee & df.empty_land & ~df.inactive]
df      = df[df.recent_penalty==0.0]
df      = df[df.school=='WYLIE']
df      = df[~df.zoning.isin(BAD_ZONING)]
col_set = set(df.columns)-{'absentee', 'empty_land', 'inactive', 'land_dict', 'property_use'}
df      = df[[*col_set]]
df      = df[(df.land_area <= max_acres) & (min_acres <= df.land_area)]

df2 = df

In [16]:
df_full = pd.concat([df1, df2], ignore_index=True)

In [17]:
df_mailing = df_to_click2mail(df_full)
df_mailing = remove_bad_owners(df_mailing)
df_mailing.shape

(333, 5)

In [18]:
timestamp = strftime("%Y%m%d_%H%M%S")
df_mailing.to_csv(f'{OUTPUT_DIR}/ml_{timestamp}.csv', index=False)

**Delinquent multi-family homes**

In [20]:
# Tax delinquent multifamily

df = pd.read_json(f'{OUTPUT_DIR}/output_taylor/output_taylor.json')
df = df[(df.property_use=='MULTIPLE RESIDENCE') & (df.recent_delinq)]
df.to_csv(f'{OUTPUT_DIR}/multi.csv', index = False)

df_mailing = df_to_click2mail(df)
df_mailing = remove_bad_owners(df_mailing)
df_mailing.to_csv(f'{OUTPUT_DIR}/ml_multi_names.csv', index=False)

**Delinquent land without penalty 1..20 acres, absentee only**

In [21]:
min_acres, max_acres = 0.9, 20.1

df      = pd.read_json(f'{OUTPUT_DIR}/output_taylor/output_taylor.json')
df      = df[good_address(df.owner_address.values)]
df      = df[df.absentee & df.empty_land & ~df.inactive ]
df      = df[(~df.recent_penalty.values.astype(bool)) & df.recent_delinq.values.astype(bool)]
df      = df[~df.zoning.isin(BAD_ZONING)]
col_set = set(df.columns)-{'absentee', 'empty_land', 'inactive', 'land_dict', 'property_use'}
df      = df[[*col_set]]
df      = df.sort_values(by='recent_penalty', ascending=False)
df      = df[(df.land_area <= max_acres) & (min_acres <= df.land_area)]

In [22]:
df_mailing         = df_to_click2mail(df)
df_mailing         = remove_bad_owners(df_mailing)
already_sent_names = pd.read_csv(f'{OUTPUT_DIR}/ml_penalty20_wylie10.csv')['Name']
df_mailing         = df_mailing[~df_mailing.Name.isin(already_sent_names.values)]

In [13]:
timestamp = strftime("%Y%m%d_%H%M%S")
df_mailing.to_csv(f'{OUTPUT_DIR}/ml_{timestamp}.csv', index=False)

**Empty lots in Hawley**

In [26]:
df      = pd.read_csv(f'{OUTPUT_DIR}/output_jones/output_jones.csv')
df      = df[df.empty_land & df.absentee]
df      = df[df.school.str.startswith('HAWLEY')]
df      = df[(df.land_area>=0.1) & (df.land_area<=5)]
df      = df[df.property_use < 'E'] # residential only
df      = remove_bad_owners(df, col='owner_name')

In [36]:
df_export = df[['prop_id', 'prop_address', 'owner_name', 'owner_address', 'property_use', 'land_area', \
                'recent_penalty', 'recent_delinq']]
df_export.to_csv(f'{OUTPUT_DIR}/hawley_empty.csv', index=False)

**Russian speaking owners**

In [53]:
df = pd.read_csv(f'{OUTPUT_DIR}/output_taylor/output_taylor.csv')
df = df[~df.owner_name.isnull()]

names = np.unique(df[df.owner_name.str.contains('^\S*(ov|ova|ev|eva|ko|ian|yan)\s', case=False)].owner_name.values)
np.unique(list(map(lambda x: x.split()[0], names)))

array(['ABAMISLIMOVA', 'ADRIAN', 'BABAIAN', 'BARBIAN', 'BATKO', 'BOTZKO',
       'BRYAN', 'BYKOV', 'CAPKO', 'CHAPKO', 'CHRISTIAN', 'CORDOVA',
       'CORPIAN', 'DAMIAN', 'GAMBLIAN', 'GORBENKO', 'HACOPIAN',
       'HARCHENKO', 'INDIAN', 'JESKO', 'JULIAN', 'KALAYDZHYAN',
       'KARABEGOV', 'KO', 'KOSHELEVA', 'KRUSHEVA', 'LANUEVA', 'LUSKO',
       'LUTZKO', 'MARDEROSIAN', 'MIKO', 'MILIAN', 'MUZECHENKO',
       'NURMUKHAMEDOV', 'OSKIERKO', 'PANIAN', 'PAYAN', 'PERMIAN',
       'PETROVA', 'PHYSICIAN', 'PLISKO', 'PRESBYTERIAN', 'QIAN', 'RHYAN',
       'ROYKO', 'RUNYAN', 'RYAN', 'SALAKO', 'SANDLIAN', 'SHAMLIAN',
       'SHOMANSUROV', 'SIKES-RYAN', 'UNITARIAN', 'VALORIAN', 'VENETIAN',
       'VILLANUEVA', 'VIVIAN', 'VOLCKO', 'ZELEV', 'ZELISKO'], dtype='<U13')

**Largest land owner**

In [12]:
AGGR_KEY = 'recent_delinq'

df    = pd.read_csv(f'{OUTPUT_DIR}/output_taylor/output_taylor.csv')
df    = df[~df.owner_name.isnull()]
df[['owner_name', AGGR_KEY]].groupby('owner_name').sum().sort_values(ascending=False, by=AGGR_KEY)[:20]

,recent_delinq
owner_name,
WELLTOWER TCG NNN LANDLORD LLC,360432.14
ER PROPCO WC LLC,238649.50
ABILENE TEACHERS FEDERAL,152068.55
WISTERIA HEALTH HOLDINGS LLC,150923.31
FUNERAL DIRECTORS LIFE INS CO,142362.80
SPT IVEY ABILENE MOB LLC,141142.33
TOWER TECH SYSTEMS INC,140908.87
WHITENER FAMILY LIMITED PTSHP 1,135691.94
PAK HARRIS ENTERPRISES LTD,125956.91
